In [60]:
import numpy as np
from scipy.spatial.transform import Rotation as R
from lab1.task2_inverse_kinematics import MetaData
from lab1.task1_forward_kinematics import *

In [13]:
l = [1, 2, 3]
l

[1, 2, 3]

In [14]:
list(reversed(l)), l

([3, 2, 1], [1, 2, 3])

In [15]:
l.pop(), l

(3, [1, 2])

In [ ]:
bvh_file_path = "../data/truncated_walk60.bvh"
joint_name, joint_parent, joint_offset = part1_calculate_T_pose(bvh_file_path)
motion_data = load_motion_data(bvh_file_path)
joint_positions, joint_orientations = part2_forward_kinematics(joint_name, joint_parent, joint_offset, motion_data, 0)
meta_data = MetaData(joint_name, joint_parent, joint_positions, '?', '?')
target_end = np.array([])

In [ ]:
path, path_names, end_to_root, fixed_to_root = meta_data
    



In [ ]:
joint_positions, joint_orientations

In [25]:
reversed(l)

In [17]:
print("hello")

hello


In [18]:
l = [2, 3, 4, 5]
len(l)

4

In [35]:
l

[2, 3, 4, 5]

In [45]:
for i in l[:-1][::1]:
    print(i)

2
3
4


In [52]:
a = np.empty((3,3), dtype=np.float64)
a

array([[1., 1., 1.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [53]:
a[0] = np.ones_like(a[0])
a

array([[1., 1., 1.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [61]:
type(R.from_quat([1, 0, 0, 0]))

scipy.spatial.transform._rotation.Rotation

In [ ]:
def joint_offsets_from_positions(joint_positions: np.ndarray,
                                 joint_parents: List[int]) -> np.ndarray:
    offsets = np.empty_like(joint_positions)
    offsets[0] = np.zeros_like(offsets[0])

    for i in range(1, joint_positions.shape[0]):
        offsets[i] = joint_positions[i] - joint_positions[joint_parents[i]]

    return offsets

In [ ]:
def update_chain_orientations(path: List[int],
                              start: int,
                              rotation,
                              joint_orientations: np.ndarray) -> None:
    """
    Update the orientations of joints from index "start" to the end effector (exclusive) in a chain of joints defined by "path".
    """
    for i in path[start:-1]:
        joint_orientations[i] = (rotation * R.from_quat(joint_orientations[i])).as_quat()

In [ ]:
from scipy.spatial.transform import Rotation as R
from lab1.task1_forward_kinematics import pose_joint_positions

def cyclic_coordinate_descent(path: List[int],
                              target_end: np.ndarray,
                              initial_joint_positions: np.ndarray,
                              joint_parents: List[int],
                              joint_positions: np.ndarray,
                              joint_orientations: np.ndarray,
                              max_iterations: int = 20,
                              max_error: float = 0.01,
                              order: int = 1) -> None:
    assert order == 1 or order == -1
    
    joint_offsets = joint_offsets_from_positions(joint_positions,
                                                 joint_parents)
    iteration_count = 0
    
    for i in path[:-1][::order]: # TODO: right now ccd only goes through joint at most once.
        if max_iterations <= iteration_count or np.linalg.norm(joint_positions[path[-1]] - target_end) <= max_error:
            break
        
        displacement = joint_positions[path[-1]] - joint_positions[i]
        target_displacement = target_end - joint_positions[i]
        # axis_of_rotation = np.cross(displacement, target_displacement)
        angle_of_rotation = np.arccos(np.dot(displacement, target_displacement) 
                                      / (np.linalg.norm(displacement) * np.linalg.norm(target_displacement)))
        rotation = R.from_rotvec(axis_of_rotation / angle_of_rotation)
        update_chain_orientations(joint_orientations, path[i:], rotation)
        joint_positions = pose_joint_positions(initial_joint_positions[0], 
                                                  joint_parents,
                                                  joint_offsets,
                                                  joint_orientations)
        
        iteration_count += 1

    # traverse each joint index in path (excluding end), in forward or backward fashion:
    # for each:
        # compute the vector from it (inclusive) to the end (exclusive)
        # compute (target_end - fixed)
        # cross product these vectors to get the axis of rotation
        # dot product these vectors to get the angle (should it be radian or degree?) of rotation.
        # compute new orientations for joints in the chain (excluding the end)
        # update the positions all joints in the chain (including the end).
        

def part1_inverse_kinematics(meta_data: MetaData,
                             joint_positions: np.ndarray,
                             joint_orientations: np.ndarray,
                             target_end: np.ndarray):
    """
    完成函数，计算逆运动学
    输入: 
        meta_data: 为了方便，将一些固定信息进行了打包，见上面的meta_data类
        joint_positions: 当前的关节位置，是一个numpy数组，shape为(M, 3)，M为关节数
        joint_orientations: 当前的关节朝向，是一个numpy数组，shape为(M, 4)，M为关节数
        target_pose: 目标位置，是一个numpy数组，shape为(3,)
    输出:
        经过IK后的姿态
        joint_positions: 计算得到的关节位置，是一个numpy数组，shape为(M, 3)，M为关节数
        joint_orientations: 计算得到的关节朝向，是一个numpy数组，shape为(M, 4)，M为关节数
    """
    path, path_names, end_to_root, fixed_to_root = meta_data.get_path_from_root_to_end()
    
    cyclic_coordinate_descent(path, 
                              target_end, 
                              meta_data.joint_initial_position, 
                              meta_data.joint_parent,
                              joint_positions, 
                              joint_orientations)

    return joint_positions, joint_orientations